==============================\
Student name: Shiqi Yin \
GitHub username: acse-sy121\
\==============================

In [ ]:
!pip install -q vtk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math  import *
import datetime, time
import pprint as pprint
import vtk
import os
os.chdir("/content/drive/MyDrive")
import vtktools
import joblib

In [ ]:
root_path = ''

times = 720
nNodes = 192060


y1 = 1.4

# --------------------------#
# -- Coordinates Fluidity --#
#---------------------------#


p1 = [4.5, y1, 1]
p2 = [5.0, y1, 1]
p3 = [5.4, y1, 1]
p4 = [5.8, y1, 1]
p5 = [6.3, y1, 1]
p6 = [6.8, y1, 1]
p7 = [7.2, y1, 1]
p8 = [7.7, y1, 1]
p9 = [6.3, 4.0, 1]
p10 = [6.3, 3.5, 1]
p11 = [6.5, 3.0, 1]
p12 = [7.0, 3.0, 1]
p13 = [6.8, 3.5, 1]
p14 = [6.8, 4.0, 1]
p15 = [4.5, 5.6, 1]
p16 = [4.5, 5.1, 1]
p17 = [4.6, 4.7, 1]
p18 = [5.0, 4.7, 1]
p19 = [5.0, 5.0, 1]
p20 = [5.0, 5.5, 1]
p21 = [8.2, 5.4, 1]
p22 = [8.2, 4.8, 1]
p23 = [8.4, 4.4, 1]
p24 = [8.8, 4.8, 1]
p25 = [8.8, 4.8, 1]
p26 = [8.8, 5.3, 1]
p27 = [7.2, 5.6, 1.5]

coordinates = np.array([p1, p2, p3, p4, p5, p6, 
             p7, p8, p9, p10, p11, p12, 
             p13, p14, p15, p16, p17, 
             p18, p19, p20, p21, p22, 
             p23, p24, p25, p26, p27])

print ('\n Coordinates Concentration ::')
print (coordinates)


 Coordinates Concentration ::
[[4.5 1.4 1. ]
 [5.  1.4 1. ]
 [5.4 1.4 1. ]
 [5.8 1.4 1. ]
 [6.3 1.4 1. ]
 [6.8 1.4 1. ]
 [7.2 1.4 1. ]
 [7.7 1.4 1. ]
 [6.3 4.  1. ]
 [6.3 3.5 1. ]
 [6.5 3.  1. ]
 [7.  3.  1. ]
 [6.8 3.5 1. ]
 [6.8 4.  1. ]
 [4.5 5.6 1. ]
 [4.5 5.1 1. ]
 [4.6 4.7 1. ]
 [5.  4.7 1. ]
 [5.  5.  1. ]
 [5.  5.5 1. ]
 [8.2 5.4 1. ]
 [8.2 4.8 1. ]
 [8.4 4.4 1. ]
 [8.8 4.8 1. ]
 [8.8 4.8 1. ]
 [8.8 5.3 1. ]
 [7.2 5.6 1.5]]


In [ ]:
#--------------------------------------------------#
#--  Function to initialise vtk files   --#
#--------------------------------------------------#
def Initialisation(filename):
    '''
      This function initialises the vtk file
      Parameters
      ----------
    
      filename : string
        The filename of the vtk file

      Returns
      ---------

      ugrid 
      
    '''
    # Read file
    print ('     Read file')
    if filename[-4:] == ".vtu":
        gridreader=vtk.vtkXMLUnstructuredGridReader()
    elif filename[-5:] == ".pvtu":
        gridreader=vtk.vtkXMLPUnstructuredGridReader()
    gridreader.SetFileName(filename)
    gridreader.Update()
    ugrid=gridreader.GetOutput()

    return ugrid

#-------------------------------------------------#
#-- Function to initialise probe filter  --#
#-------------------------------------------------#
def InitialisePointData(ugrid, coordinates):

    # Initialise probe
    points = vtk.vtkPoints()
    points.SetDataTypeToDouble()

    # Create points to be extracted
    print ('     Gathering  coordinates')
    NrbPoints = 0
    for nodeID in range(len(coordinates)):
        NrbPoints += 1
        points.InsertNextPoint(coordinates[nodeID][0], coordinates[nodeID][1], coordinates[nodeID][2])

    print ('           Set points into data...')
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    probe = vtk.vtkProbeFilter()

    print ('           Map data into probe...', 'VTK version ::', vtk.vtkVersion.GetVTKMajorVersion(),'.', vtk.vtkVersion.GetVTKMinorVersion())
    if vtk.vtkVersion.GetVTKMajorVersion() <= 5:
        probe.SetInput(polydata)
        probe.SetSource(ugrid)
    else:
        probe.SetInputData(polydata)
        probe.SetSourceData(ugrid)

    probe.Update()
    
    return probe, points, NrbPoints
#-------------------------------------------------#
#-- Function to initialise cell filter           -#
#-------------------------------------------------#
def InitialiseCellLocator(ugrid):

    # Initialise locator
    print ('     Initialise cell Locator')
    CellLocator = vtk.vtkCellLocator()
    CellLocator.SetDataSet(ugrid)
    CellLocator.Update()
    
    return CellLocator

In [ ]:
#---------------------------------------------------------------------
# EXTRACT DATA
#---------------------------------------------------------------------
def load_coord_data(filename, fields_list):
    
    #-- Fields
    FIELDS = []

    for f in range(len(fields_list)):
        tic = time.time()
        #-- CO2
        CO2     = []
        #-- Time
        TimeVTU = []
        r = 0
        CO2.append([])
        fieldname = fields_list[f]
        # print(fieldname)
        # Read file
        ugrid = Initialisation(filename)
        # Initialise probe
        probe, points, NrbPoints = InitialisePointData(ugrid, coordinates)
        # Initialise cell location
        CellLocator = InitialiseCellLocator(ugrid)
        #-- Check Validity of points
        # print ('           Check point Validity')
        valid_ids = probe.GetOutput().GetPointData().GetArray('vtkValidPointMask')
        validPoints = vtktools.arr([valid_ids.GetTuple1(i) for i in range(NrbPoints)])
        # print ('           ... ', len(validPoints)-np.sum(validPoints), 'invalid points')
        # Extract data
        # print ('     Extract Data')
        for nodeID in range(len(coordinates)):
            # If valid point, extract using probe,
            # Otherwise extract the cell:
            #    If no cell associated - then it is really a non-valid point outside the domain
            #    Otherwise: do the average over the cell values - this provide the tracer value.
            # We need to do that as it is a well-known bug in vtk libraries - sometimes it returns an invalid node while it is not...
            if validPoints[nodeID] == 1:
                tmp = probe.GetOutput().GetPointData().GetArray(fieldname).GetTuple(nodeID)
                CO2[r].append(tmp)
            else:
                coord_tmp = np.array(points.GetPoint(nodeID))
                cellID =  CellLocator.FindCell(coord_tmp) # cell ID which contains the node
                idlist=vtk.vtkIdList()
                ugrid.GetCellPoints(cellID, idlist)
                pointsID_to_cellID = np.array([idlist.GetId(k) for k in range(idlist.GetNumberOfIds())]) # give all the points asociated with this cell
                if len(pointsID_to_cellID) == 0: # Non-valid points - We assign negative value - like that we know we are outside the domain
                    CO2[r].append(-1e20)
                else:
                    tmp = 0
                    for pointID in pointsID_to_cellID:
                        tmp += ugrid.GetPointData().GetArray(fieldname).GetTuple(pointID)[0]
                    tmp = tmp/len(pointsID_to_cellID)
                    CO2[r].append(tmp)
            # print("nodeID: ", valid_ids)

        # Time
        # time_tmp = probe.GetOutput().GetPointData().GetArray('Time').GetValue(0)
        # TimeVTU.append(time_tmp)
        # print ('      Time ::', time_tmp)
        # print ('      CO2  ::', CO2[r])

        r += 1

        # print ('\n Time in (s) ::')
        # print (TimeVTU)    
        # np.savetxt('TimeVTU.dat',TimeVTU)

        print ('\n CO2 ::')
        CO2 = np.array(CO2)
        # print (CO2)
        # CO2_all = [np.array(TimeVTU)]
        # CO2_all = np.transpose(np.append(CO2_all, np.transpose(CO2), axis = 0))

        # joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_410.pkl') #CO2 data -> ( nSnapshot x 7(sensors) )
        # joblib.dump(CO2_all, '/content/gdrive/MyDrive/ACSE-9/models/CO2_Fluidity_410.pkl') #with the time column
        # joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_all.pkl') #CO2 data -> ALL time levels

        toc = time.time()

        # print ('\n\nTime : ', toc - tic, 'sec')

        FIELDS.append(CO2)

    FIELDS = np.array(FIELDS)

    return FIELDS


In [ ]:
# The path where the vtu files are located
path = '/content/drive/MyDrive/Cotrace_fixed_final/' 
# path = '/content/drive/MyDrive/output_files/vtu_files/case_1/'
# path = '/content/drive/MyDrive/output_files/vtu_files/case_2/'
# The prefix of the file name of the vtu file
name_simu = 'output'

fields_list = ['Tracer', 'Virus1']

In [ ]:
vtu_start = 577 #361
vtu_end   = 720
vtu_step  = 1


results = []
for t in range(vtu_start, vtu_end+1, vtu_step):
    filename = path + 'Cotrace_fixed_{}'.format(t) + '.vtu'
    # filename = path + 'output_{}'.format(t) + '.vtu'
    print(filename)
    result = load_coord_data(filename, fields_list)
    results.append(result)
results_array = np.array(results)


# results_array = results_array.reshape(120, 5, 18)

In [ ]:
print(results_array.shape)

(144, 2, 1, 27, 1)


In [ ]:
results_final = results_array[:,:,0,:,0]

In [ ]:
results_final.shape

(144, 2, 27)

Prediction data of CO2 concentration level and viral load for case 1

In [ ]:
# np.save('/content/drive/MyDrive/Pred_data.npy',results_final)

Real data of CO2 concentration level and viral load for case 1

In [ ]:
# np.save('/content/drive/MyDrive/CFD_data.npy',results_final)

Prediction data of CO2 concentration level and viral load for Scenario 2

In [ ]:
# np.save('/content/drive/MyDrive/output_files/DATA/Human_prediction/Case_2/Pred_data.npy',results_final)

Real data of CO2 concentration level and viral load for Scenario 2

In [ ]:
np.save('/content/drive/MyDrive/output_files/DATA/Human_prediction/Case_2/CFD_data.npy',results_final)